# Overview

In this tutorial we will introduce the new Tensorflow 2.0 / Keras API, as well as basic concepts related with neural networks.

**Tensorflow 2.0 API**

* creating models: sequential and functional API
* creating optimizers
* creating loss functions
* algorithm training

**Multilayer Perceptron (MLPs)**

* matrix multiplication
* activation functions

**Convolutional Neural Networks (CNNs)**

* convolutional operations
* spatial downsampling

*Advanced Topics*

* batch normalization
* weight regularization
* advanced activation functions
* custom loss functions
* custom metrics

# Tensorflow 2.0 API

Tensorflow is a free and open-source software library developed by the Google Brain team for dataflow and differentiable programming across a range of tasks. It is a symbolic math library, and is most popularly used for machine learning applications such as neural networks. In November 2019, the first stable release of the verson 2.0 library was made available, with significant changes including:

* formal integration of the high-level Keras API for easy model building
* `eager execution` of code, eliminating the need to manually compile an abstract syntax tree using a `session.run()` call
* improved support for model deployment in production on any platform
* improved support for distributed machine learning paradigms

More information highlighting the key improvements can be found here: https://www.tensorflow.org/guide/effective_tf2

## Import

In this tutorial we will use the following Numpy and Tensorflow library components:

In [ ]:
import numpy as np
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers

## Overview of Keras

To develop a model using the Tensorflow/Keras API, we need to define two key objects: a Keras `model` and a Keras `layer`. After a `model` has been created with multiple `layers` the model needs to be *compiled* prior to algorithm *training*. In the following sections, we will introduce these key concepts then show how to instatiate and use these objects in Python. 

**NOTE**: We will be introducing *concepts* in these tutorial without any formal dataset for algorithm training and evalulation. See subsequent tutorials for more detailed information and guides for training specific neural network architectures on various datasets. 

### Keras models 

A Keras `model` is a high-level object that encapsulates and organizes one or multiple Keras `layers`. There are two main types of models available in Keras: the `Sequential` model, and the `Model` class used with the functional API.

All Keras models have a number of methods and attributes in common:

* `model.layers` is a flattened list of the layers comprising the model
* `model.inputs` is the list of input tensors of the model
* `model.outputs` is the list of output tensors of the model
* `model.summary()` prints a summary representation of your model

In addition there are a number of key methods used to pass data through the model during training and inference:

* `model.fit()` is used to train a model with data
* `model.evaluate()` is used to evaluate moel performance
* `model.predict()` is used to pass new data to a trained network

More information can be found here: https://keras.io/models/about-keras-models/

### Keras layers

A Keras `layer` is a `callable` Python object that represents functionality for a single layer in a neural network model. All Keras layers have a number of methods in common:

* `layer.get_weights()` returns the weights of the layer as a list of Numpy arrays
* `layer.set_weights(weights)` sets the weights of the layer from a list of Numpy arrays (with the same shapes as the output of get_weights)
* `layer.get_config()` returns a dictionary containing the configuration of the layer

In [ ]:
# --- Example of instantiating a new Keras layer object
layer = layers.Dense(32)

# --- Example of invoking a common layer method to get configurations
print(type(layer))
print(layer.get_config())

More information can be found here: https://keras.io/layers/about-keras-layers/

## Creating Models

As described above, there are two primary ways to create a model using Tensorflow/Keras: the `Sequential` model and the functional API using the `Model` class. For maxmimum flexibility, we will use the functional API throughout all tutorials, however we will also demonstrate the `Sequential` model here for completeness.

### Sequential Model

The `Sequential` model allows you to define simple architectures layer-by-layer in a *sequential* manner. However this approach is limited in that it does not allow you to create models that share layers or have multiple inputs or outputs. Nonetheless for a conventional feed-forward neural network, this approach may be sufficient.

In [ ]:
# --- Option #1: add layer objects in list
model = models.Sequential([
    layers.Dense(32, input_shape=(784,)),
    layers.ReLU(),
    layers.Dense(10),
    layers.Softmax()])

In [ ]:
# --- Option #2: add layer objects using the add() method
model = models.Sequential()
model.add(layers.Dense(32, input_shape=(784,)))
model.add(layers.ReLU())
model.add(layers.Dense(10))
model.add(layers.Softmax())

In [ ]:
# --- Print summary of model architecture
model.summary()

### Functional API

The functional API allows for a broader flexibility to model definition. To use, simply define an arbitrary graph structure by passing layers into one another until the entire network has been templated. Then, select one or multiple input(s) and one or multiple output(s) to pass as arguments into the `Model` class constructor.

In [ ]:
# --- Define graph by passing layers into one another
inputs = Input(shape=(784,))
x = layers.Dense(32)(inputs)
x = layers.ReLU()(x)
x = layers.Dense(10)(x)
x = layers.Softmax()(x)

# --- Define model by passing input(s) and output(s)
model = Model(inputs=inputs, outputs=x)

In [ ]:
# --- Print summary of model architecture
model.summary()

## Compiling a Model

Now that our model has been defined, let us prepare our model for training. To do so we will need to formally **compile** the graph and define the training process using several key components (each represented by Keras Python objects):

* loss function
* optimizer
* metric(s) (optional)

### Defining a loss object

Keras has a number of loss functions encapsulated by Python classes in the `tf.losses.*` module. The most commonly used include:

* categorical cross entopy (classification tasks)
* mean absolute or squared errors (regressions tasks)
* Huber loss (many box algorithms)

**IMPORTANT**: if you are training a classification task and your last model layer *does not* include an activation function (e.g. it represents raw logit scores) you must use the `from_logits=True` flag when defining you loss.

In [ ]:
# --- Define a MAE loss
loss = losses.MeanAbsoluteError()

# --- Define a categorical cross-entropy loss
loss = losses.SparseCategoricalCrossentropy()

### Defining an optimizer object

Keras has a number of optimizer functions encapsulated by Python classes in the `tf.optimizers.*` module. The most commonly used include:

* stochastic gradient descent
* SGD + momentum
* Adam (recommended by default)

To instantiate a new optimizer object, simply pass the learning rate into the class constructor.

In [ ]:
# --- Define an SGD optimizer
optimizer = optimizers.SGD(learning_rate=2e-4)

# --- Define an Adam optimizer
optimizer = optimizers.Adam(learning_rate=2e-4)

### Defining a metric

While losses are used by a neural network to guide optimization of model parameters, metrics are used by a human to gauage model performance in a more easily interpretable way. The most common metric for classification tasks is overall model `accuracy` (%). Other custom metrics can be defined as shown in advanced sections below.

### Compiling

Once the model `optimizer`, `loss` and `metric` have been defined, simply pass these objects into the `model.compile()` method to prepare for training:

In [ ]:
# --- Compile model
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=['accuracy'])

The model is now compiled and ready for training! See subsequent tutorials for more detaisl.

## Saving and Loading a Model

After a model has been successfully trained, it can be saved and/or loaded by simply using the `model.save()` and `models.load_model()` methods. Note that any custom losses and/or metrics will need to be provided via a dictionary.

In [ ]:
# --- Serialize a model
model.save('./intro.hdf5')

In [ ]:
# --- Load a serialized model
del model
model = models.load_model('./intro.hdf5')

In [ ]:
# --- Delete saved model
import os
os.remove('./intro.hdf5')